## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-20-46-01 +0000,startribune,Hegseth cites 'fog of war' in defending follow...,https://www.startribune.com/hegseth-cites-fog-...
1,2025-12-02-20-45-31 +0000,nyt,Russian Launch Site Mishap Leaves Country’s Sp...,https://www.nytimes.com/2025/12/01/science/rus...
2,2025-12-02-20-44-44 +0000,nyt,Putin and Witkoff Meet in Russia as U.S. Pushe...,https://www.nytimes.com/2025/12/02/world/europ...
3,2025-12-02-20-44-40 +0000,nyt,University of Oklahoma Instructor on Leave Aft...,https://www.nytimes.com/2025/12/02/us/oklahoma...
4,2025-12-02-20-41-52 +0000,nypost,Defiant Maduro dances as deportation flights t...,https://nypost.com/2025/12/02/world-news/defia...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,60
54,new,20
136,out,14
610,house,13
165,up,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-12-02-19-27-00 +0000,wsj,An ICE operation will target Minneapolis’s Som...,https://www.wsj.com/politics/policy/ice-plans-...,134
284,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,131
254,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,127
340,2025-12-01-21-10-44 +0000,nypost,White House doesn’t rule out boots on the grou...,https://nypost.com/2025/12/01/us-news/trump-do...,124
150,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...,121


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,134,2025-12-02-19-27-00 +0000,wsj,An ICE operation will target Minneapolis’s Som...,https://www.wsj.com/politics/policy/ice-plans-...
298,72,2025-12-01-23-32-00 +0000,nypost,White House says admiral ordered second strike...,https://nypost.com/2025/12/01/us-news/wh-says-...
272,57,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...
131,53,2025-12-02-15-55-11 +0000,nypost,Vladimir Putin says he’s ready to go to war wi...,https://nypost.com/2025/12/02/world-news/putin...
39,49,2025-12-02-19-52-00 +0000,wsj,D.C. National Guard Shooting Suspect Pleads No...,https://www.wsj.com/us-news/dc-national-guard-...
104,41,2025-12-02-17-13-25 +0000,nyt,"Juan Orlando Hernández, Former President of Ho...",https://www.nytimes.com/2025/12/02/us/politics...
65,41,2025-12-02-19-06-42 +0000,nypost,Apple shakes up AI team as top exec John Giann...,https://nypost.com/2025/12/02/business/apple-s...
261,39,2025-12-02-02-43-18 +0000,nypost,DHS chief Kristi Noem wants full travel ban on...,https://nypost.com/2025/12/01/us-news/dhs-chie...
179,37,2025-12-02-11-42-00 +0000,wsj,Defense Secretary Pete Hegseth is facing one o...,https://www.wsj.com/politics/policy/hegseth-un...
256,36,2025-12-02-03-13-14 +0000,nypost,Trump calls Tennessee special election a ‘big ...,https://nypost.com/2025/12/01/us-news/trump-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
